In [17]:
import pyodbc
import requests
import pandas as pd

# Connection string
# Enters your database 'SERVER' name:
conn_str = 'DRIVER={ODBC Driver 18 for SQL Server};' \
           'SERVER=QUOCBUI-PERSONA\\MSSQLSERVER01;' \
           'DATABASE=FireFixDB;' \
           'Connection Timeout=300;' \
           'Login Timeout=300;' \
           'LongAsMax=yes;' \
           'TrustServerCertificate=yes;' \
           'Trusted_Connection=yes;'

def print_raw_file_links_by_changeset_hash(input_changeset_hash):
    global conn_str
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    try:
        cursor.execute('''  
            SELECT cf.Previous_File_Name,
                cf.Updated_File_Name,
                c.Mercurial_Type,
                cf.Changeset_Hash_ID,
                c.Parent_Hashes
            FROM Changeset_Files cf
            INNER JOIN Changeset_Details c ON c.Hash_Id = cf.Changeset_Hash_ID
            WHERE cf.Changeset_Hash_ID = ?;
        ''', input_changeset_hash)

        records = cursor.fetchall()
    finally:
        cursor.close()
        conn.close()

    link_format = "https://hg.mozilla.org/{mercurial_type}/raw-file/{changeset_hash_id}/{file_path}"

    def generate_file_link(mercurial_type, changeset_hash_id, file_path):
        return link_format.format(
            mercurial_type=mercurial_type,
            changeset_hash_id=changeset_hash_id,
            file_path=file_path[2:]  # Remove "a/" or "b/" prefixes
        )

    count = 1
    for record in records:
        prev_file, updated_file, mercurial_type, changeset_hash, parent_hash = record
        
        mercurial_types = mercurial_type.split(' | ')
        print(f"File {str(count)} - Previous File: '{prev_file}' && Updated File: '{updated_file}'")
        print(f"File {str(count)} - List of Raw File Links for Previous File:")
        if prev_file != '/dev/null':
            for mercurial in mercurial_types:
                print(f"[{mercurial}]: {generate_file_link(mercurial, changeset_hash, prev_file)}")
        else:
            print("- No available")

        print(f"File {str(count)} - List of Raw File Links for Updated File:")
        if updated_file != '/dev/null':
            for mercurial in mercurial_types:
                print(f"[{mercurial}]: {generate_file_link(mercurial, changeset_hash, updated_file)}")
        else:
            print("- No available")
        print("\n")
        count += 1

    if len(records) == 0:
        print(f"No files associated with the changeset: {input_changeset_hash}")


def extract_file_content(changeset_hash_id, mercurial_type, fully_qualified_filename):
    sanitized_filename = fully_qualified_filename[2:] if fully_qualified_filename.startswith(("a/", "b/")) else fully_qualified_filename

    link_format = "https://hg.mozilla.org/{mercurial_type}/raw-file/{changeset_hash_id}/{file_path}"
    url = link_format.format(
        mercurial_type=mercurial_type,
        changeset_hash_id=changeset_hash_id,
        file_path=sanitized_filename
    )

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch file content from {url}: {e}")
        return None


def get_file_type_distribution():
    global conn_str

    query = '''
    -- Get file type statistics
    SELECT 
        File_Type,
        Total_Count,
        ROUND((CAST(Total_Count AS FLOAT) / (SELECT COUNT(*) FROM Changeset_Files WHERE 
            (Previous_File_Name LIKE '%.js' OR Previous_File_Name LIKE '%.py' OR Previous_File_Name LIKE '%.c' OR Previous_File_Name LIKE '%.cpp')
            OR 
            (Updated_File_Name LIKE '%.js' OR Updated_File_Name LIKE '%.py' OR Updated_File_Name LIKE '%.c' OR Updated_File_Name LIKE '%.cpp')
        )) * 100, 2) AS Percentage
    FROM (
        SELECT 
            'JavaScript (.js)' AS File_Type, 
            COUNT(*) AS Total_Count
        FROM Changeset_Files
        WHERE 
            (Previous_File_Name LIKE '%.js' OR Updated_File_Name LIKE '%.js')
        UNION ALL
        SELECT 
            'Python (.py)' AS File_Type, 
            COUNT(*) AS Total_Count
        FROM Changeset_Files
        WHERE 
            (Previous_File_Name LIKE '%.py' OR Updated_File_Name LIKE '%.py')
        UNION ALL
        SELECT 
            'C (.c)' AS File_Type, 
            COUNT(*) AS Total_Count
        FROM Changeset_Files
        WHERE 
            (Previous_File_Name LIKE '%.c' OR Updated_File_Name LIKE '%.c')
        UNION ALL
        SELECT 
            'C++ (.cpp)' AS File_Type, 
            COUNT(*) AS Total_Count
        FROM Changeset_Files
        WHERE 
            (Previous_File_Name LIKE '%.cpp' OR Updated_File_Name LIKE '%.cpp')
    ) AS File_Stats
    ORDER BY Percentage DESC;
    '''

    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        try:
            cursor.execute(query)
            records = cursor.fetchall()  # Fetch all records
            columns = [column[0] for column in cursor.description]  # Extract column names
            df = pd.DataFrame.from_records(records, columns=columns)  # Convert to DataFrame
        finally:
            cursor.close()
        conn.close()
        return df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def get_changeset_files(changeset_hash):
    global conn_str

    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        try:
            cursor.execute('''
                SELECT
                    File_Status,
                    Previous_File_Name, 
                    Updated_File_Name
                FROM Changeset_Files
                WHERE Changeset_Hash_ID = ?;
                ''', (changeset_hash,))
            records = cursor.fetchall()
        finally:
            cursor.close()
        conn.close()

        if not records:
            print(f"No records found for Changeset Hash ID: {changeset_hash}")
            return None
        else:
            df = pd.DataFrame.from_records(
                records, 
                columns=["File Status", "Previous File Name", "Updated File Name"]
            )
            return df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def get_list_of_changesets(bug_id):
    global conn_str

    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        try:
            cursor.execute('''
                SELECT DISTINCT
                    Changeset_Hash_ID,
                    Changeset_Summary,
                    Mercurial_Type
                FROM Changeset_Bug_Mapping m
                INNER JOIN Changeset_Details c ON c.Hash_ID = m.Changeset_Hash_ID
                WHERE Bug_ID = ?;
                ''', (bug_id,))
            records = cursor.fetchall()
        finally:
            cursor.close()
        conn.close()

        if not records:
            print(f"No records found for Changeset Bug ID: {bug_id}")
            return None
        else:
            df = pd.DataFrame.from_records(
                records, 
                columns=["Changeset_Hash_ID", "Changeset_Summary", "Mercurial_Type"]
            )
            return df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


How to formulate raw file URL links based on a specific changeset?

In [18]:
# Print out the list of raw file links based on the input changeset hash
changeset_hash_id = "c6be663fa1946241927de6d960e152f5c1a1f8f7"
print_raw_file_links_by_changeset_hash(changeset_hash_id)

File 1 - Previous File: 'a/mozglue/build/WindowsDllBlocklist.cpp' && Updated File: 'b/mozglue/build/WindowsDllBlocklist.cpp'
File 1 - List of Raw File Links for Previous File:
[releases/mozilla-b2g44_v2_5]: https://hg.mozilla.org/releases/mozilla-b2g44_v2_5/raw-file/c6be663fa1946241927de6d960e152f5c1a1f8f7/mozglue/build/WindowsDllBlocklist.cpp
[releases/mozilla-beta]: https://hg.mozilla.org/releases/mozilla-beta/raw-file/c6be663fa1946241927de6d960e152f5c1a1f8f7/mozglue/build/WindowsDllBlocklist.cpp
File 1 - List of Raw File Links for Updated File:
[releases/mozilla-b2g44_v2_5]: https://hg.mozilla.org/releases/mozilla-b2g44_v2_5/raw-file/c6be663fa1946241927de6d960e152f5c1a1f8f7/mozglue/build/WindowsDllBlocklist.cpp
[releases/mozilla-beta]: https://hg.mozilla.org/releases/mozilla-beta/raw-file/c6be663fa1946241927de6d960e152f5c1a1f8f7/mozglue/build/WindowsDllBlocklist.cpp




How to extract a specific file content?

In [19]:
# Define changeset hash and mercurial type:
changeset_hash_id = "c6be663fa1946241927de6d960e152f5c1a1f8f7"
mercurial_type = "releases/mozilla-beta"

# Example of print out file content:
fully_qualified_filename_a = "a/browser/base/content/nsContextMenu.js"
file_content_a = extract_file_content(changeset_hash_id, mercurial_type, fully_qualified_filename_a)
print(file_content_a) # Print out file content


# Example of saving file content to .txt:
fully_qualified_filename_b = "b/browser/base/content/nsContextMenu.js"
file_content_b = extract_file_content(changeset_hash_id, mercurial_type, fully_qualified_filename_b)
if file_content_b:
    # Simplify the filename for saving
    simplified_filename = fully_qualified_filename_b.replace("/", "_").replace("\\", "_")
    output_path = f"./{simplified_filename}.txt"

    # Write to file
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(file_content_b)
        print(f"Content saved to {output_path}")
else:
    print("Failed to fetch file content.")

/* vim: set ts=2 sw=2 sts=2 et tw=80: */
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at http://mozilla.org/MPL/2.0/.

Components.utils.import("resource://gre/modules/PrivateBrowsingUtils.jsm");
Components.utils.import("resource://gre/modules/InlineSpellChecker.jsm");
Components.utils.import("resource://gre/modules/LoginManagerContextMenu.jsm");
Components.utils.import("resource://gre/modules/BrowserUtils.jsm");
Components.utils.import("resource://gre/modules/XPCOMUtils.jsm");
Components.utils.import("resource://gre/modules/Services.jsm");


XPCOMUtils.defineLazyModuleGetter(this, "CustomizableUI",
  "resource:///modules/CustomizableUI.jsm");
XPCOMUtils.defineLazyModuleGetter(this, "Pocket",
  "resource:///modules/Pocket.jsm");
XPCOMUtils.defineLazyModuleGetter(this, "LoginHelper",
  "resource://gre/modules/LoginHelper.jsm");

var gContextMenuContentData = null;

fu

How do I get the distribution of file types in the dataset?

In [20]:
# Print out File Type Distribution:
df = get_file_type_distribution()
df.index = range(1, len(df) + 1)
df.style.set_caption("File Type Distribution").format({'Percentage': "{:.2f}%"})

,File_Type,Total_Count,Percentage
1,C++ (.cpp),444217,57.49%
2,JavaScript (.js),262328,33.95%
3,C (.c),40384,5.23%
4,Python (.py),25833,3.34%


How to get the list of files associated with a specific changeset/commit?

In [21]:
changeset_hash = "c15b2a5abf1ca5c4169ef6340be56a25b5ec4f45"
df = get_changeset_files(changeset_hash)

if df is not None:
    df.index = range(1, len(df) + 1)
    styled_df = df.style.set_caption(f"Files in Changeset: {changeset_hash}").set_properties(**{'text-align': 'left'}).set_table_styles([
        {'selector': 'thead th', 'props': [('text-align', 'left')]}
    ])
    display(styled_df)  # Use IPython's display function to render in Jupyter Notebook

,File Status,Previous File Name,Updated File Name
1,new,/dev/null,b/toolkit/mozapps/update/tests/unit_service_updater/invalidArgWorkingDirPathLocalUNCFailureSvc_win.js
2,new,/dev/null,b/toolkit/mozapps/update/tests/unit_service_updater/invalidArgInstallDirPathTooLongFailureSvc.js
3,modified,a/toolkit/mozapps/update/tests/unit_base_updater/marAppApplyUpdateSuccess.js,b/toolkit/mozapps/update/tests/unit_base_updater/marAppApplyUpdateSuccess.js
4,new,/dev/null,b/toolkit/mozapps/update/tests/unit_base_updater/invalidArgStageDirNotInInstallDirFailure_win.js
5,new,/dev/null,b/toolkit/mozapps/update/tests/unit_base_updater/invalidArgPatchDirPathTraversalFailure.js
6,new,/dev/null,b/toolkit/mozapps/update/tests/unit_base_updater/invalidArgWorkingDirPathRelativeFailure.js
7,new,/dev/null,b/toolkit/mozapps/update/tests/unit_service_updater/invalidArgInstallDirPathTraversalFailureSvc.js
8,new,/dev/null,b/toolkit/mozapps/update/tests/unit_service_updater/invalidArgInstallWorkingDirPathNotSameFailureSvc_win.js
9,modified,a/toolkit/mozapps/update/tests/unit_base_updater/marAppInUseStageFailureComplete_win.js,b/toolkit/mozapps/update/tests/unit_base_updater/marAppInUseStageFailureComplete_win.js
10,modified,a/toolkit/mozapps/update/tests/TestAUSHelper.cpp,b/toolkit/mozapps/update/tests/TestAUSHelper.cpp


How do I retrieve the list of changesets associated with a specific bug?

In [23]:
bug_id = "1496458"

df = get_list_of_changesets(bug_id)
if df is not None:
    df.index = range(1, len(df) + 1)
    styled_df = df.style.set_caption(f"Changesets in Bug Id: {bug_id}").set_properties(**{'text-align': 'left'}).set_table_styles([
        {'selector': 'thead th', 'props': [('text-align', 'left')]}
    ])
    display(styled_df)

,Changeset_Hash_ID,Changeset_Summary,Mercurial_Type
1,00269c0edf56d1a920d1b0dfd757ae9221c40e30,Patrick Brosset - Bug 1496458 - Display the flex-container accordion first if a container+item is selected in markup-view; r=gl,integration/autoland | mozilla-central
2,f5bb9d00011e09fbd12b922c51dbd258fd746b4b,Gabriel Luong - Bug 1496458 - Show the parent flex container accordion at the top if it exists. r=mtigley,mozilla-central
